# Modeling Cas9 PID through RBM

In [1]:
import sys, os, re, random, warnings, subprocess, time
sys.path.append(os.path.dirname(os.getcwd()))
warnings.filterwarnings("ignore")

from itertools import product
from random import shuffle

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from torchpgm.model import *
from torchpgm.edge import Edge
from torchpgm.layers import *
from torchpgm.graphic import draw_G
from torchpgm.data import RBMDataWithPAM, RBMData

from sklearn.metrics import * 
from config import *
from utils import *
from utils.pam import *

In [ ]:
device = "cuda"
folder = f"{DATA}/cas9/vink"

batch_size = 300
Nh = 200
Npam = 5
n_epochs = 1000
start_supervision_epoch = 20
l1b = 0.0025
l2 = 0.05
lambda_pa = 0.00
lr = 0.0001

#l1b = .5
#l2 = 0.1

model_full_name = f"rbmssl_pid_h{Nh}_npam{Npam}"

## The Standard RBM

We load the data. More details about how to build these data file are provided in the appropriated notebook

In [ ]:
train_dataset = RBMData(f"{folder}/data.pt", subset = "train")
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

val_dataset = RBMData(f"{folder}/data.pt", subset = "val")
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

Starting point for the potentials

In [ ]:
q_pi, N_pi = train_dataset[0][0].size()

g_pi = torch.zeros(q_pi, N_pi)
for x_pi, _, w, _ in train_dataset:
    g_pi += w* x_pi
g_pi = np.log(1+g_pi)
W = sum(train_dataset.weights)
g_pi = (g_pi - g_pi.mean(0)[None]).flatten()/W

We load the model

In [ ]:
model_full_name = f"rbm_pid_h{Nh}_npam{Npam}"

pi = OneHotLayer(None, N=N_pi, q=q_pi, name="pi")
h = GaussianLayer(N=Nh, name="hidden")

E = [(pi.name, h.name)]
E.sort()

model_rbm = PI_RBM(layers= {pi.name: pi, h.name: h}, edges=E, name = model_full_name)
optimizer = optim.AdamW(model_rbm.parameters(), lr=lr)

model_rbm = model_rbm.to(device)
#best_epoch = 180
#model_rbm.load(f"{folder}/weights/{model_full_name}_{best_epoch}.h5")
model_rbm.ais()

We train the model

In [ ]:
for epoch in range(1, n_epochs):
    model_rbm.train_epoch(optimizer, train_loader, train_dataset, 
                          regularizer = {"l1b" : l1b, "l2": l2, "l1b_pam": 0}, 
                          epoch = epoch, savepath=f"{folder}/weights/{model_full_name}")
    shuffle(train_dataset.x_m)

    if not epoch % 5:
        model_rbm.ais()
        model_rbm.val(val_loader, visible_layers, hidden_layers, epoch)

We evaluate the model

In [2]:
model_rbm.eval()
model_rbm.ais()

print("Training set")
model_rbm.val(train_loader, visible_layers, hidden_layers, 0)

print("Validation set")
model_rbm.val(val_loader, visible_layers, hidden_layers, 0)

model_rbm = model_rbm.to("cpu")
device = "cpu"

NameError: name 'model_rbm' is not defined

## The Semi-Supervised Learning RBM

We load the data. More details about how to build these data file are provided in the appropriated notebook

In [6]:
train_dataset = RBMData(f"{folder}/data.pt", subset = "train")    
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

val_dataset = RBMData(f"{folder}/data.pt", subset = "val")
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

train_dataset_labelled = RBMDataWithPAM(f"{folder}/data.pt", Npam, subset = "train_labelled")
train_loader_labelled = DataLoader(train_dataset_labelled, batch_size = batch_size, shuffle = True)

val_dataset_labelled = RBMDataWithPAM(f"{folder}/data.pt", Npam, subset = "val_labelled")
val_loader_labelled = DataLoader(val_dataset_labelled, batch_size = batch_size, shuffle = False)

Available :  x pi_seq y L labelled subset weights
Available :  x pi_seq y L labelled subset weights
Available :  x pi_seq y L labelled subset weights
Available :  x pi_seq y L labelled subset weights


Starting point for the potentials

In [7]:
q_pi, N_pi = train_dataset[0][0].size()
g_pi = torch.zeros(q_pi, N_pi)
for x_pi, _, w, _ in train_dataset:
    g_pi += w* x_pi
g_pi = np.log(1+g_pi)
W = sum(train_dataset.weights)
g_pi = (g_pi - g_pi.mean(0)[None]).flatten()/W

We load the model

In [ ]:
device = "cuda"

pi = OneHotLayer(g_pi, N=N_pi, q=q_pi, name="pi")
h = GaussianLayer(N=Nh, name="hidden")

classifier = PAM_classifier(Nh, Npam * 4, dropout = 0.)
E = [(pi.name, h.name)]
E.sort()

gamma = 5

model_rbm_ssl = PI_RBM_SSL(classifier, layers= {pi.name: pi, h.name: h}, edges=E, name = "")
optimizer = optim.AdamW(model_rbm_ssl.parameters(), lr=lr)
model_rbm_ssl = model_rbm_ssl.to(device)

We train the model using both training set :
- The set of unlabelled sequences
- The set of labelled sequences

In [ ]:
for epoch in range(1, 181):
    model_rbm_ssl.train_epoch_classifier(optimizer, train_loader, train_loader_labelled, train_dataset, train_dataset_labelled, 
                                     regularizer = {"l1b" : l1b, "l2":l2 , "l1b_pam": 0}, 
                                     epoch = epoch, savepath=f"{folder}/weights/{model_full_name}")
    shuffle(train_dataset_labelled.x_m)
    shuffle(train_dataset.x_m)

    if not epoch % 5:
        model_rbm_ssl.ais()
        model_rbm_ssl.val(val_loader, visible_layers, hidden_layers, epoch)
        model_rbm_ssl.val_classifier(val_loader_labelled, visible_layers, hidden_layers, epoch)

In [ ]:
model_rbm_ssl.eval()
model_rbm_ssl.ais()

print("Training set")
model_rbm_ssl.val(train_loader, visible_layers, hidden_layers, 0)
model_rbm_ssl.val_classifier(train_loader_labelled, visible_layers, hidden_layers, 0)

print("Validation set")
model_rbm_ssl.val(val_loader, visible_layers, hidden_layers, 0)
model_rbm_ssl.val_classifier(val_loader_labelled, visible_layers, hidden_layers, 0)

model_rbm = model_rbm.to("cpu")
device = "cpu"

## Study on the effect of gamma on the quality of the model

For each value of $\gamma$, we train one SSL-RBM on the same training and validation sets. Training also possible using the script `train_all_rbm.py`

In [ ]:
for gamma in gammas:
    torch.cuda.empty_cache()
    train_dataset = RBMData(f"{folder}/data.pt", subset = "train" )
    train_loader = DataLoader(train_dataset, batch_size = 100, shuffle = True, drop_last=True)

    val_dataset = RBMData(f"{folder}/data.pt", subset = "val")
    val_loader = DataLoader(val_dataset, batch_size = 300, shuffle = False)

    train_dataset_labelled = RBMDataWithPAM(f"{folder}/data.pt", Npam, subset = "train_labelled")
    train_loader_labelled = DataLoader(train_dataset_labelled, batch_size = 11, shuffle = True, drop_last=True)

    val_dataset_labelled = RBMDataWithPAM(f"{folder}/data.pt", Npam, subset = "val_labelled")
    val_loader_labelled = DataLoader(val_dataset_labelled, batch_size = 300, shuffle = False)
    
    q_pi, N_pi = train_dataset[0][0].size()
    g_pi = torch.zeros(q_pi, N_pi)
    for x_pi, _, w, _ in train_dataset:
        g_pi += w* x_pi
    g_pi = np.log(1+g_pi)
    W = sum(train_dataset.weights)
    g_pi = (g_pi - g_pi.mean(0)[None]).flatten()/W
    
    device = "cuda"
    folder = f"{DATA}/cas9/vink"
    model_full_name = f"rbmssl_pid_h{Nh}_npam{Npam}"    
    visible_layers = ["pi"]
    hidden_layers = ["hidden"]

    pi = OneHotLayer(None, N=N_pi, q=q_pi, name="pi")
    h = GaussianLayer(N=Nh, name="hidden")
    classifier = PAM_classifier(Nh, Npam * 4, dropout = 0.)
    E = [(pi.name, h.name)]
    E.sort()

    model_rbm = PI_RBM_SSL(classifier, layers= {pi.name: pi, h.name: h}, edges=E, gamma = gamma, name = f"classifier_{model_full_name}")
    optimizer = optim.AdamW(model_rbm.parameters(), lr=lr)
    model_rbm = model_rbm.to("cuda")
    model_rbm.ais()
    
    for epoch in range(1, 181):
        model_rbm.train_epoch_classifier(optimizer, train_loader, train_loader_labelled, train_dataset, train_dataset_labelled, 
                                         regularizer = {"l1b" : l1b, "l2":l2 , "l1b_pam": 0}, 
                                         epoch = epoch, savepath=f"{folder}/weights/c{model_full_name}")
        shuffle(train_dataset_labelled.x_m)
        shuffle(train_dataset.x_m)

        if not epoch % 5:
            model_rbm.ais()
            model_rbm.val(val_loader, visible_layers, hidden_layers, epoch)
            model_rbm.val_classifier(val_loader_labelled, visible_layers, hidden_layers, epoch)

Available :  x pi_seq y L labelled subset weights
Available :  x pi_seq y L labelled subset weights
Available :  x pi_seq y L labelled subset weights
Available :  x pi_seq y L labelled subset weights
Train SSL Epoch: 1 [100%] || Time: 0 min 1 || Loss: 0.659 || P(v): -3.276 || Reg: 0.637 || PI: 0.051  || PAM: 0.530
Train SSL Epoch: 2 [100%] || Time: 0 min 1 || Loss: 0.497 || P(v): -3.181 || Reg: 0.523 || PI: 0.062  || PAM: 0.535
Train SSL Epoch: 3 [100%] || Time: 0 min 0 || Loss: 0.309 || P(v): -2.976 || Reg: 0.432 || PI: 0.096  || PAM: 0.576
Train SSL Epoch: 4 [100%] || Time: 0 min 0 || Loss: 0.072 || P(v): -2.632 || Reg: 0.360 || PI: 0.191  || PAM: 0.591
Train SSL Epoch: 5 [100%] || Time: 0 min 0 || Loss: -0.234 || P(v): -2.118 || Reg: 0.304 || PI: 0.392  || PAM: 0.642
Val Epoch: 5 [100%] || Time: 0 min 0 || P(v): -1.808 || P(v,h): -1.944 || PI: 0.503
Val SSL Epoch: 5 [100%] || Time: 0 min 0 || P(v): -1.888 || P(v,h): -2.027 || PI: 0.496 || PAM: 0.699
Train SSL Epoch: 6 [100%] || Time

Train SSL Epoch: 106 [100%] || Time: 0 min 1 || Loss: 0.003 || P(v): -0.382 || Reg: 0.023 || PI: 0.737  || PAM: 0.883
Train SSL Epoch: 107 [100%] || Time: 0 min 1 || Loss: 0.008 || P(v): -0.376 || Reg: 0.023 || PI: 0.740  || PAM: 0.847
Train SSL Epoch: 108 [100%] || Time: 0 min 0 || Loss: 0.003 || P(v): -0.384 || Reg: 0.023 || PI: 0.739  || PAM: 0.892
Train SSL Epoch: 109 [100%] || Time: 0 min 0 || Loss: 0.003 || P(v): -0.381 || Reg: 0.023 || PI: 0.739  || PAM: 0.869
Train SSL Epoch: 110 [100%] || Time: 0 min 1 || Loss: 0.007 || P(v): -0.387 || Reg: 0.023 || PI: 0.739  || PAM: 0.858
Val Epoch: 110 [100%] || Time: 0 min 0 || P(v): -0.391 || P(v,h): -0.525 || PI: 0.734
Val SSL Epoch: 110 [100%] || Time: 0 min 0 || P(v): -0.512 || P(v,h): -0.641 || PI: 0.733 || PAM: 0.767
Train SSL Epoch: 111 [100%] || Time: 0 min 0 || Loss: -0.002 || P(v): -0.386 || Reg: 0.023 || PI: 0.740  || PAM: 0.889
Train SSL Epoch: 112 [100%] || Time: 0 min 0 || Loss: 0.009 || P(v): -0.385 || Reg: 0.023 || PI: 0.74

Train SSL Epoch: 31 [100%] || Time: 0 min 1 || Loss: 0.100 || P(v): 1.274 || Reg: 0.068 || PI: 0.695  || PAM: 0.519
Train SSL Epoch: 32 [100%] || Time: 0 min 1 || Loss: 0.087 || P(v): 1.149 || Reg: 0.066 || PI: 0.694  || PAM: 0.493
Train SSL Epoch: 33 [100%] || Time: 0 min 1 || Loss: 0.088 || P(v): 1.031 || Reg: 0.065 || PI: 0.694  || PAM: 0.501
Train SSL Epoch: 34 [100%] || Time: 0 min 1 || Loss: 0.083 || P(v): 0.911 || Reg: 0.063 || PI: 0.694  || PAM: 0.499
Train SSL Epoch: 35 [100%] || Time: 0 min 0 || Loss: 0.092 || P(v): 0.791 || Reg: 0.062 || PI: 0.694  || PAM: 0.536
Val Epoch: 35 [100%] || Time: 0 min 0 || P(v): 1.448 || P(v,h): 1.312 || PI: 0.697
Val SSL Epoch: 35 [100%] || Time: 0 min 0 || P(v): 1.253 || P(v,h): 1.120 || PI: 0.688 || PAM: 0.522
Train SSL Epoch: 36 [100%] || Time: 0 min 1 || Loss: 0.075 || P(v): 1.390 || Reg: 0.061 || PI: 0.694  || PAM: 0.537
Train SSL Epoch: 37 [100%] || Time: 0 min 1 || Loss: 0.089 || P(v): 1.283 || Reg: 0.059 || PI: 0.695  || PAM: 0.504
Trai

Train SSL Epoch: 136 [100%] || Time: 0 min 0 || Loss: -0.024 || P(v): -0.268 || Reg: 0.035 || PI: 0.750  || PAM: 0.817
Train SSL Epoch: 137 [100%] || Time: 0 min 1 || Loss: -0.038 || P(v): -0.258 || Reg: 0.035 || PI: 0.752  || PAM: 0.790
Train SSL Epoch: 138 [100%] || Time: 0 min 0 || Loss: -0.019 || P(v): -0.262 || Reg: 0.035 || PI: 0.751  || PAM: 0.750
Train SSL Epoch: 139 [100%] || Time: 0 min 0 || Loss: -0.012 || P(v): -0.259 || Reg: 0.035 || PI: 0.752  || PAM: 0.770
Train SSL Epoch: 140 [100%] || Time: 0 min 0 || Loss: -0.021 || P(v): -0.258 || Reg: 0.035 || PI: 0.751  || PAM: 0.766
Val Epoch: 140 [100%] || Time: 0 min 0 || P(v): -0.277 || P(v,h): -0.414 || PI: 0.745
Val SSL Epoch: 140 [100%] || Time: 0 min 0 || P(v): -0.418 || P(v,h): -0.545 || PI: 0.751 || PAM: 0.692
Train SSL Epoch: 141 [100%] || Time: 0 min 0 || Loss: -0.027 || P(v): -0.259 || Reg: 0.035 || PI: 0.751  || PAM: 0.793
Train SSL Epoch: 142 [100%] || Time: 0 min 0 || Loss: -0.007 || P(v): -0.262 || Reg: 0.035 || PI

Train SSL Epoch: 61 [100%] || Time: 0 min 1 || Loss: 0.022 || P(v): 0.339 || Reg: 0.041 || PI: 0.699  || PAM: 0.670
Train SSL Epoch: 62 [100%] || Time: 0 min 1 || Loss: 0.023 || P(v): 0.306 || Reg: 0.040 || PI: 0.700  || PAM: 0.674
Train SSL Epoch: 63 [100%] || Time: 0 min 1 || Loss: 0.022 || P(v): 0.271 || Reg: 0.040 || PI: 0.700  || PAM: 0.652
Train SSL Epoch: 64 [100%] || Time: 0 min 0 || Loss: 0.016 || P(v): 0.239 || Reg: 0.040 || PI: 0.700  || PAM: 0.662
Train SSL Epoch: 65 [100%] || Time: 0 min 0 || Loss: 0.016 || P(v): 0.208 || Reg: 0.039 || PI: 0.701  || PAM: 0.671
Val Epoch: 65 [100%] || Time: 0 min 0 || P(v): 0.219 || P(v,h): 0.085 || PI: 0.702
Val SSL Epoch: 65 [100%] || Time: 0 min 0 || P(v): 0.079 || P(v,h): -0.055 || PI: 0.696 || PAM: 0.675
Train SSL Epoch: 66 [100%] || Time: 0 min 1 || Loss: 0.011 || P(v): 0.192 || Reg: 0.039 || PI: 0.702  || PAM: 0.667
Train SSL Epoch: 67 [100%] || Time: 0 min 1 || Loss: 0.012 || P(v): 0.166 || Reg: 0.039 || PI: 0.702  || PAM: 0.679
Tra

...

Train SSL Epoch: 77 [100%] || Time: 0 min 0 || Loss: -0.008 || P(v): 0.236 || Reg: 0.031 || PI: 0.722  || PAM: 0.792
Train SSL Epoch: 78 [100%] || Time: 0 min 1 || Loss: -0.010 || P(v): 0.215 || Reg: 0.031 || PI: 0.724  || PAM: 0.832
Train SSL Epoch: 79 [100%] || Time: 0 min 1 || Loss: -0.009 || P(v): 0.188 || Reg: 0.031 || PI: 0.725  || PAM: 0.835
Train SSL Epoch: 80 [100%] || Time: 0 min 1 || Loss: -0.007 || P(v): 0.161 || Reg: 0.031 || PI: 0.726  || PAM: 0.806
Val Epoch: 80 [100%] || Time: 0 min 0 || P(v): 0.163 || P(v,h): 0.027 || PI: 0.724
Val SSL Epoch: 80 [100%] || Time: 0 min 0 || P(v): 0.025 || P(v,h): -0.111 || PI: 0.721 || PAM: 0.792
Train SSL Epoch: 81 [100%] || Time: 0 min 0 || Loss: -0.004 || P(v): 0.140 || Reg: 0.031 || PI: 0.728  || PAM: 0.811
Train SSL Epoch: 82 [100%] || Time: 0 min 0 || Loss: 0.003 || P(v): 0.116 || Reg: 0.031 || PI: 0.728  || PAM: 0.828
Train SSL Epoch: 83 [100%] || Time: 0 min 1 || Loss: 0.002 || P(v): 0.085 || Reg: 0.030 || PI: 0.728  || PAM: 0.82

Available :  x pi_seq y L labelled subset weights
Available :  x pi_seq y L labelled subset weights
Available :  x pi_seq y L labelled subset weights
Train SSL Epoch: 1 [100%] || Time: 0 min 0 || Loss: 0.647 || P(v): -3.271 || Reg: 0.636 || PI: 0.052  || PAM: 0.534
Train SSL Epoch: 2 [100%] || Time: 0 min 1 || Loss: 0.471 || P(v): -3.167 || Reg: 0.524 || PI: 0.064  || PAM: 0.555
Train SSL Epoch: 3 [100%] || Time: 0 min 0 || Loss: 0.253 || P(v): -2.951 || Reg: 0.435 || PI: 0.102  || PAM: 0.581
Train SSL Epoch: 4 [100%] || Time: 0 min 0 || Loss: -0.035 || P(v): -2.591 || Reg: 0.365 || PI: 0.201  || PAM: 0.585
Train SSL Epoch: 5 [100%] || Time: 0 min 0 || Loss: -0.403 || P(v): -2.074 || Reg: 0.309 || PI: 0.405  || PAM: 0.600
Val Epoch: 5 [100%] || Time: 0 min 0 || P(v): -1.760 || P(v,h): -1.896 || PI: 0.516
Val SSL Epoch: 5 [100%] || Time: 0 min 0 || P(v): -1.850 || P(v,h): -1.981 || PI: 0.507 || PAM: 0.700
Train SSL Epoch: 6 [100%] || Time: 0 min 0 || Loss: -0.870 || P(v): -1.375 || Reg:

Train SSL Epoch: 106 [100%] || Time: 0 min 1 || Loss: -0.009 || P(v): -0.358 || Reg: 0.026 || PI: 0.737  || PAM: 0.859
Train SSL Epoch: 107 [100%] || Time: 0 min 0 || Loss: -0.009 || P(v): -0.355 || Reg: 0.026 || PI: 0.739  || PAM: 0.837
Train SSL Epoch: 108 [100%] || Time: 0 min 0 || Loss: -0.003 || P(v): -0.350 || Reg: 0.026 || PI: 0.740  || PAM: 0.838
Train SSL Epoch: 109 [100%] || Time: 0 min 0 || Loss: 0.004 || P(v): -0.350 || Reg: 0.026 || PI: 0.740  || PAM: 0.813
Train SSL Epoch: 110 [100%] || Time: 0 min 0 || Loss: -0.013 || P(v): -0.357 || Reg: 0.026 || PI: 0.740  || PAM: 0.866
Val Epoch: 110 [100%] || Time: 0 min 0 || P(v): -0.364 || P(v,h): -0.502 || PI: 0.735
Val SSL Epoch: 110 [100%] || Time: 0 min 0 || P(v): -0.492 || P(v,h): -0.632 || PI: 0.730 || PAM: 0.799
Train SSL Epoch: 111 [100%] || Time: 0 min 1 || Loss: -0.004 || P(v): -0.358 || Reg: 0.026 || PI: 0.742  || PAM: 0.841
Train SSL Epoch: 112 [100%] || Time: 0 min 1 || Loss: 0.000 || P(v): -0.356 || Reg: 0.026 || PI: 

Train SSL Epoch: 159 [100%] || Time: 0 min 0 || Loss: 0.001 || P(v): -0.313 || Reg: 0.027 || PI: 0.756  || PAM: 0.891
Train SSL Epoch: 160 [100%] || Time: 0 min 0 || Loss: 0.007 || P(v): -0.313 || Reg: 0.027 || PI: 0.756  || PAM: 0.850
Val Epoch: 160 [100%] || Time: 0 min 0 || P(v): -0.329 || P(v,h): -0.461 || PI: 0.749
Val SSL Epoch: 160 [100%] || Time: 0 min 0 || P(v): -0.464 || P(v,h): -0.601 || PI: 0.753 || PAM: 0.791
Train SSL Epoch: 161 [100%] || Time: 0 min 0 || Loss: 0.006 || P(v): -0.303 || Reg: 0.027 || PI: 0.757  || PAM: 0.864
Train SSL Epoch: 162 [100%] || Time: 0 min 0 || Loss: 0.003 || P(v): -0.312 || Reg: 0.027 || PI: 0.757  || PAM: 0.883
Train SSL Epoch: 163 [100%] || Time: 0 min 1 || Loss: 0.006 || P(v): -0.308 || Reg: 0.027 || PI: 0.757  || PAM: 0.874
Train SSL Epoch: 164 [100%] || Time: 0 min 0 || Loss: -0.002 || P(v): -0.304 || Reg: 0.027 || PI: 0.757  || PAM: 0.893
Train SSL Epoch: 165 [100%] || Time: 0 min 1 || Loss: 0.001 || P(v): -0.306 || Reg: 0.027 || PI: 0.75

Train SSL Epoch: 31 [100%] || Time: 0 min 0 || Loss: 0.075 || P(v): 1.534 || Reg: 0.044 || PI: 0.696  || PAM: 0.760
Train SSL Epoch: 32 [100%] || Time: 0 min 0 || Loss: 0.072 || P(v): 1.434 || Reg: 0.043 || PI: 0.695  || PAM: 0.733
Train SSL Epoch: 33 [100%] || Time: 0 min 1 || Loss: 0.066 || P(v): 1.345 || Reg: 0.042 || PI: 0.696  || PAM: 0.815
Train SSL Epoch: 34 [100%] || Time: 0 min 1 || Loss: 0.063 || P(v): 1.261 || Reg: 0.041 || PI: 0.696  || PAM: 0.759
Train SSL Epoch: 35 [100%] || Time: 0 min 1 || Loss: 0.063 || P(v): 1.187 || Reg: 0.040 || PI: 0.697  || PAM: 0.796
Val Epoch: 35 [100%] || Time: 0 min 0 || P(v): 1.219 || P(v,h): 1.083 || PI: 0.699
Val SSL Epoch: 35 [100%] || Time: 0 min 0 || P(v): 1.048 || P(v,h): 0.908 || PI: 0.693 || PAM: 0.802
Train SSL Epoch: 36 [100%] || Time: 0 min 0 || Loss: 0.061 || P(v): 1.166 || Reg: 0.039 || PI: 0.697  || PAM: 0.754
Train SSL Epoch: 37 [100%] || Time: 0 min 0 || Loss: 0.058 || P(v): 1.094 || Reg: 0.039 || PI: 0.697  || PAM: 0.777
Trai

## Model Validation

In [10]:
best_epoch = 360
model_rbm.load(f"{folder}/weights/{model_full_name}_{best_epoch}.h5")

In [69]:
model_rbm.eval()
model_rbm.ais()

4.912878036499023

In [14]:
model_rbm.val(train_loader, visible_layers, hidden_layers, 0)
model_rbm.val_classifier(train_loader_labelled, visible_layers, hidden_layers, 0)

Val Epoch: 0 [100%] || Time: 0 min 0 || P(v): -0.503 || P(v,h): -0.638 || PI: 0.712
Val SSL Epoch: 0 [100%] || Time: 0 min 0 || P(v): -0.583 || P(v,h): -0.717 || PI: 0.722 || PAM: 0.889


In [15]:
model_rbm.val(val_loader, visible_layers, hidden_layers, 0)
model_rbm.val_classifier(val_loader_labelled, visible_layers, hidden_layers, 0)

Val Epoch: 0 [100%] || Time: 0 min 0 || P(v): -0.603 || P(v,h): -0.737 || PI: 0.707
Val SSL Epoch: 0 [100%] || Time: 0 min 0 || P(v): -0.930 || P(v,h): -1.055 || PI: 0.731 || PAM: 0.985


In [72]:
model_rbm = model_rbm.to("cpu")
device = "cpu"